In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, cluster
import matplotlib.pyplot as plt
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

import os, pickle

from proj_util import cal_avg_precision

FOLDER_PATH = ''


In [2]:
test_set = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','test_set.p'),'rb'))

user_mapping = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','user_mapping.p'),'rb'))
product_mapping = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','product_mapping.p'),'rb'))

In [3]:
test_set.user_id = test_set.user_id.map(user_mapping)
test_set.product_id = test_set.product_id.map(product_mapping)

In [8]:
model1_results = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','result_model1_1300.pickle'),'rb'))

V = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','final-nmf-prior-V.p'),'rb'))
U = pickle.load(open(os.path.join(FOLDER_PATH,'pickle','final-nmf-prior-U.p'),'rb'))

In [10]:
for i in range(len(model1_results)):
    model1_results[i][0] = user_mapping[model1_results[i][0]]
    model1_results[i][1] = [product_mapping[x] for x in model1_results[i][1]]

In [46]:
def get_final_recommendation(model1_result, nmf2_result):
    '''
    Final Recommendation = Model 1 Top 5 + Model 2 Top 3 New Products
    '''
    model1_product_count = 5
    model1_products = model1_result[:model1_product_count]
    
    model2_product_count = 3
    new_products = [x for x in nmf2_result if x not in model1_result]
    model2_products = new_products[:model1_product_count + model2_product_count - len(model1_products)]

    return model1_products + model2_products

In [47]:
scores = []
for u, model1_result in model1_results:
    user_item_score = np.matmul(U[u:u+1], V.T)[0]
    model2_result = np.argsort(user_item_score)[-1:-11:-1]
    
    final_recommendation = get_final_recommendation(model1_result,model2_result)

    gt_product = test_set.query('user_id=={}'.format(u)).product_id.values
    score = cal_avg_precision(gt_product, final_recommendation)
    scores.append(score)

In [48]:
np.mean(scores) # 0.4732402014652014

0.4732402014652014